In [1]:
import sys
sys.path.insert(0, '../../')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Union
import pandas as pd
from IPython.display import display, Markdown

import candle
import experiments.textgenutils as gutils

## (1) Initialize Model with Pre-trained Weights

In [ ]:
# One of ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl']
#    gpt2:         124M params
#    gpt2-medium:  354M params
#    gpt2-large:   774M params
#    gpt2-xl:    1,557M params

model = candle.models.gpt.GPT.from_pretrained('gpt2-large')

## (2) Compare Quality of Sampling Techniques

In [4]:
SAMPLING_TECHNIQUES = [
    # (technique, (beam_size, top_k, top_p))
    ('Beam Size = 2, P = 0.95, Top K = 100', (3, 100, 0.95)),
    ('P = 0.95, Top K = 100', (1, 100, 0.95)),
    ('Nucleus Sampling P = 0.95', (1, None, 0.95)),
    ('Top K = 40', (1, 40, None)),
    ('Beam Size = 10', (10, None, None)),
    ('Pure Sampling', (1, None, None)),
]


def compare_responses_of_sampling_techniques(prompt: str,
                                             n_tokens_to_generate: int = 50,
                                             trials: int = 2,
                                             sampling_techniques: list = SAMPLING_TECHNIQUES):
    tokenizer = candle.models.gpt.GPT2BPETokenizer()

    responses_by_config_df = pd.DataFrame(columns=range(trials), index=[i[0] for i in sampling_techniques])

    for config in sampling_techniques:
        for trial in range(trials):
            (label, (beam_size, top_k, top_p)) = config
            
            model.clear_kv_cache()
            generator = gutils.generate_text(model, tokenizer, prompt,
                                             n_tokens_to_generate=n_tokens_to_generate,
                                             beam_size=beam_size,
                                             top_k=top_k,
                                             top_p=top_p,
                                             use_kv_cache=True)

            responses_by_config_df.loc[label, trial] = ''.join(list(generator))

    responses_by_config_df = responses_by_config_df.stack().reset_index().set_index(['level_0', 'level_1'])
    responses_by_config_df.index.names = ['Sampling Technique', 'Trial']
    responses_by_config_df.columns = ['Response']

    prompt_df = pd.DataFrame([prompt], columns=[''], index=['Prompt'])
    properties = {'text-align': 'left', 'white-space': 'pre-wrap'}
    display(prompt_df.style.set_properties(**properties))
    display(responses_by_config_df.style.set_properties(**properties))

In [5]:
compare_responses_of_sampling_techniques(
    'Once upon a time, there was a cat whose name was Maukoo. He loves eating and cuddling. '
)

,
Prompt,"Once upon a time, there was a cat whose name was Maukoo. He loves eating and cuddling."


In [6]:
compare_responses_of_sampling_techniques(
    'It\'s Black Friday, and Min wants to buy a new ski jacket. However, she doesn\'t know '
    'what to buy. '
)

,
Prompt,"It's Black Friday, and Min wants to buy a new ski jacket. However, she doesn't know what to buy."


In [7]:
compare_responses_of_sampling_techniques(
    'In the heart of a whimsical forest, where trees whispered secrets to the wind and '
    'flowers danced in the sunlight, there lived a tiny ladybug named Bella. '
)

,
Prompt,"In the heart of a whimsical forest, where trees whispered secrets to the wind and flowers danced in the sunlight, there lived a tiny ladybug named Bella."


In [8]:
compare_responses_of_sampling_techniques(
    'A hush fell over the crowded auditorium as the maestro raised his baton, signaling '
    'the commencement of the symphony. '
)

,
Prompt,"A hush fell over the crowded auditorium as the maestro raised his baton, signaling the commencement of the symphony."


In [9]:
compare_responses_of_sampling_techniques(
    'Justine grew up in Alaska, and worked alongside her family on a small commercial '
    'salmon fishing boat during the summers until she graduated from college. '
)

,
Prompt,"Justine grew up in Alaska, and worked alongside her family on a small commercial salmon fishing boat during the summers until she graduated from college."


In [10]:
compare_responses_of_sampling_techniques(
    'A team of scientists has made an astonishing discovery in the depths of the Amazon '
    'Rainforest – a new species of giant turtle. '
)

,
Prompt,A team of scientists has made an astonishing discovery in the depths of the Amazon Rainforest – a new species of giant turtle.


In [11]:
compare_responses_of_sampling_techniques(
"""In the hush of twilight's gentle embrace,
Where shadows dance and stars find their place,
A symphony of silence fills the air,
As nature's lullaby begins to prepare.

The day's vibrant hues gently fade away,
Replaced by a canvas of silver and gray,"""
)

,
Prompt,"In the hush of twilight's gentle embrace, Where shadows dance and stars find their place, A symphony of silence fills the air, As nature's lullaby begins to prepare. The day's vibrant hues gently fade away, Replaced by a canvas of silver and gray,"


In [12]:
compare_responses_of_sampling_techniques(
    """[FADE IN]

EXT. NEW YORK CITY STREETS - NIGHT

A bustling metropolis, New York City comes alive under the cloak of darkness. Skyscrapers 
pierce the sky, their windows glowing like a constellation of stars.

Amidst this urban jungle, a lone figure walks with a determined stride. This is ANNA, a young 
woman in her early twenties, with an air of mystery about her. """
)

,
Prompt,"[FADE IN] EXT. NEW YORK CITY STREETS - NIGHT A bustling metropolis, New York City comes alive under the cloak of darkness. Skyscrapers pierce the sky, their windows glowing like a constellation of stars. Amidst this urban jungle, a lone figure walks with a determined stride. This is ANNA, a young woman in her early twenties, with an air of mystery about her."
